In [21]:
from langchain_community.document_loaders import PyPDFLoader,WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq
from langchain.chains import create_retrieval_chain
from datetime import datetime,date
from langchain_core.prompts import ChatPromptTemplate
import json
groq_api_key = "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"

In [22]:
user_input="give the information of the axis bank for last 10 days stocks"
user_prompt= f"""
Extract from {user_input} and return in JSON format only the key features of the provided input. This should include the name of the company, the starting date, and the last date in DD-MM-YYYY format. Also, include the timeframe, which can be one of the following: day, 1 month, 3 months, 6 months, or year. Additionally, provide the number of companies and any other details that would be useful for making a data request to a URL.
"""
user_prompt

'\nExtract from give the information of the axis bank for last 10 days stocks and return in JSON format only the key features of the provided input. This should include the name of the company, the starting date, and the last date in DD-MM-YYYY format. Also, include the timeframe, which can be one of the following: day, 1 month, 3 months, 6 months, or year. Additionally, provide the number of companies and any other details that would be useful for making a data request to a URL.\n'

In [23]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot giving the information realted to indian stock market referencing {date}. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "Always return JSON. Do not include any other text or formatting characters."),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="MarketMinds", user_input=user_prompt,date=date.today())
messages

[SystemMessage(content='You are a helpful AI bot giving the information realted to indian stock market referencing 2024-07-25. Your name is MarketMinds.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content='Always return JSON. Do not include any other text or formatting characters.'),
 HumanMessage(content='\nExtract from give the information of the axis bank for last 10 days stocks and return in JSON format only the key features of the provided input. This should include the name of the company, the starting date, and the last date in DD-MM-YYYY format. Also, include the timeframe, which can be one of the following: day, 1 month, 3 months, 6 months, or year. Additionally, provide the number of companies and any other details that would be useful for making a data request to a URL.\n')]

In [24]:
llm = ChatGroq(
    groq_api_key=groq_api_key, model_name="llama3-groq-70b-8192-tool-use-preview"
)
json_res=json.loads(llm.invoke(user_prompt).content)
json_res

{'company': 'Axis Bank',
 'startDate': '2023-09-01',
 'endDate': '2023-09-10',
 'timeFrame': '10 days',
 'numberOfCompanies': 1,
 'url': 'https://api.stockmarket.com/axisbank/stocks?startDate=2023-09-01&endDate=2023-09-10&timeFrame=10%20days&numberOfCompanies=1'}

In [25]:
loader = WebBaseLoader(
    [
        "https://api.upstox.com/v2/historical-candle/NSE_EQ%7CINE040A01034/day/2024-07-23/2024-07-01"
        # "https://newsapi.org/v2/everything?q=(NSE%20AND%20BSE)&apiKey=d1858fd8650743deb697aea90617d602",
        # "https://newsapi.org/v2/everything?domains=moneycontrol.com,cnbc.com,bloomberg.com,thehindubusinessline.com/topic/nse,nseindia.com/resources/exchange-communication-media-center&apiKey=d1858fd8650743deb697aea90617d602",
    ]
)
web_data = loader.load()
web_data

[Document(metadata={'source': 'https://api.upstox.com/v2/historical-candle/NSE_EQ%7CINE040A01034/day/2024-07-23/2024-07-01'}, page_content='{"status":"success","data":{"candles":[["2024-07-23T00:00:00+05:30",1647.55,1647.8,1605,1618.4,26001076,0],["2024-07-22T00:00:00+05:30",1617,1651,1612.65,1642.55,26872411,0],["2024-07-19T00:00:00+05:30",1616,1623,1603.5,1607.3,13001076,0],["2024-07-18T00:00:00+05:30",1608,1623.95,1599.15,1614.8,16858532,0],["2024-07-16T00:00:00+05:30",1620.75,1627,1616.8,1619.75,14475162,0],["2024-07-15T00:00:00+05:30",1623.5,1629.8,1615.2,1622.1,16687847,0],["2024-07-12T00:00:00+05:30",1622,1638.4,1611.15,1622.7,28024980,0],["2024-07-11T00:00:00+05:30",1623.95,1625.85,1601,1621.9,22088537,0],["2024-07-10T00:00:00+05:30",1629.95,1640,1620.75,1626.1,22753591,0],["2024-07-09T00:00:00+05:30",1625,1646.7,1620.35,1636.5,24275439,0],["2024-07-08T00:00:00+05:30",1645.3,1654.95,1627.15,1635.35,19543900,0],["2024-07-05T00:00:00+05:30",1685,1685,1642.2,1648.1,41121274,0],["2

In [26]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
web_splitted=text_splitter.split_documents(web_data)
web_splitted

[Document(metadata={'source': 'https://api.upstox.com/v2/historical-candle/NSE_EQ%7CINE040A01034/day/2024-07-23/2024-07-01'}, page_content='{"status":"success","data":{"candles":[["2024-07-23T00:00:00+05:30",1647.55,1647.8,1605,1618.4,26001076,0],["2024-07-22T00:00:00+05:30",1617,1651,1612.65,1642.55,26872411,0],["2024-07-19T00:00:00+05:30",1616,1623,1603.5,1607.3,13001076,0],["2024-07-18T00:00:00+05:30",1608,1623.95,1599.15,1614.8,16858532,0],["2024-07-16T00:00:00+05:30",1620.75,1627,1616.8,1619.75,14475162,0],["2024-07-15T00:00:00+05:30",1623.5,1629.8,1615.2,1622.1,16687847,0],["2024-07-12T00:00:00+05:30",1622,1638.4,1611.15,1622.'),
 Document(metadata={'source': 'https://api.upstox.com/v2/historical-candle/NSE_EQ%7CINE040A01034/day/2024-07-23/2024-07-01'}, page_content='24-07-12T00:00:00+05:30",1622,1638.4,1611.15,1622.7,28024980,0],["2024-07-11T00:00:00+05:30",1623.95,1625.85,1601,1621.9,22088537,0],["2024-07-10T00:00:00+05:30",1629.95,1640,1620.75,1626.1,22753591,0],["2024-07-09T0

In [27]:
embeddings_obj=OllamaEmbeddings(model='llava-phi3:3.8b')
db=FAISS.from_documents(web_splitted,embeddings_obj)
db

In [28]:
prompt = ChatPromptTemplate.from_template(
"""
Given the context below and today's date {date} (which is a {day}), answer the following questions as accurately as possible.
with some fluctuation in percentage values of params with respect to today
Context:
<context>
{context}
</context>
Questions : {input}
""" 
)
document_chain = create_stuff_documents_chain(llm, prompt)

In [29]:
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [30]:
prompt="how much deviation in % can be there for each candle values can be occur respect to today for next 5 days give it respective dates"

In [31]:
response=retrieval_chain.invoke(input={'input':prompt,
                        'context':web_splitted,
                        'date':date.today(),
                        'day':datetime.today().strftime("%A")
                        })
print(response['answer'])

To calculate the deviation in percentage for each candle value with respect to today, we first need to find the difference between the future candle values and today's value. Then, we can divide this difference by today's value and multiply by 100 to get the percentage deviation.

Let's do this for the next five days:

1. 2024-07-26: 
   - Open: 1647.55
   - High: 1647.8
   - Low: 1605
   - Close: 1618.4
   - Volume: 26001076

   Deviation in %:
   - Open: ((1647.55 - 1618.4) / 1618.4) * 100 = 1.81%
   - High: ((1647.8 - 1618.4) / 1618.4) * 100 = 1.83%
   - Low: ((1605 - 1618.4) / 1618.4) * 100 = -0.83%
   - Close: ((1618.4 - 1618.4) / 1618.4) * 100 = 0%

2. 2024-07-27:
   - Open: 1617
   - High: 1651
   - Low: 1612.65
   - Close: 1642.55
   - Volume: 26872411

   Deviation in %:
   - Open: ((1617 - 1618.4) / 1618.4) * 100 = -0.07%
   - High: ((1651 - 1618.4) / 1618.4) * 100 = 2.05%
   - Low: ((1612.65 - 1618.4) / 1618.4) * 100 = -0.35%
   - Close: ((1642.55 - 1618.4) / 1618.4) * 100 =